# Load and index the oceanographic dataset of global ocean profiles CORA

Data are available through the  E.U. Copernicus Marine Environment Monitoring Service (http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=INSITU_GLO_TS_OA_REP_OBSERVATIONS_013_002_b).

### Short description of the CORA dataset
The In Situ delayed mode product designed for reanalysis purposes integrates the best available version of in situ data for temperature and salinity measurements. These data are collected from main global networks (Argo, GOSUD, OceanSITES, World Ocean Database) completed by European data provided by EUROGOOS regional systems and national system by the regional INS TAC components. It is updated on a yearly basis. 

Here we are using the version of the dataset used This version is a merged product between the previous version of CORA and EN4 distributed by the Met Office for the period 1950-1990.

### Description of the Notebook
**This script needs the archived data to be already downloaded locally (edit _root_ variable)**
This script call a function XXXXXX which extract the yearly archives, read the netcdfs, create dataframe, store the dataframe into csv. 

The prerequisite for executing this notebook is to download the CORA dataset in a directory *myCORAdir* that can be access by the notebook.



In [1]:
import os
#import sys
import tarfile

import xarray as xr
import pandas as pd
import numpy as np

### Path to CORA database

In [1]:
# path to CORA database
root='/Volumes/ADATA/Database/COPERNICUS_CORA/INSITU_GLO_TS_OA_REP_OBSERVATIONS_013_002_b/CORIOLIS-GLOBAL-CORA-OBS_FULL_TIME_SERIE/data'

# directory to be read (each directory/year correspond to a specific tar archive)
yyyy = '2018' 

Here, we load the data from the CORA databse (http://www.coriolis.eu.org/Data-Products/Products/CORA) which are already donwloaded locally.

More specifically, we use the profiles used in the Objective Analysis product which are gridded on the same depth level (http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=INSITU_GLO_TS_OA_REP_OBSERVATIONS_013_002_b)

## Function to convert CORA netcdf file to python Dataframe
-> extract the intitial archive, read the netcdf, create a dataframe, store the dataframe into csv. 

Code adapted from Kelvin Balem's repository (https://github.com/quai20/DataViz)

### Extraction of archive CORA-5.2-data-YYYY.tgz in individual directory YYYY

In [3]:
tgzfile = 'CORA-5.2-data-'+yyyy+'.tgz'
tardir = root+'/'+yyyy

# Create the target directory if it doesnt exist
if not os.path.exists(tardir):
    os.makedirs(tardir)
    
# Extract tar    
tar = tarfile.open(root+'/'+tgzfile)
tar.extractall(path=tardir)
tar.close()


In [4]:
# Load test Netcdf to display structure of the file
examplefile_temp='2018/OA_CORA5.2_20180115_dat_PSAL.nc'
with xr.open_dataset(root+'/'+examplefile_temp,decode_times=False) as ds:
    print(ds.keys())
print(ds.head())
a=ds['DEPH']

KeysView(<xarray.Dataset>
Dimensions:              (N_LEVELS: 152, N_PROF: 43462)
Dimensions without coordinates: N_LEVELS, N_PROF
Data variables:
    REFERENCE_DATE_TIME  |S16 ...
    DATA_TYPE            |S16 ...
    PLATFORM_NUMBER      (N_PROF) |S8 ...
    WMO_INST_TYPE        (N_PROF) |S4 ...
    DC_REFERENCE         (N_PROF) |S32 ...
    JULD                 (N_PROF) float64 ...
    LATITUDE             (N_PROF) float64 ...
    LONGITUDE            (N_PROF) float64 ...
    DEPH                 (N_LEVELS) float64 ...
    PSAL_PROC            (N_PROF) int8 ...
    PSAL_QC              (N_PROF, N_LEVELS) float32 ...
    PSAL                 (N_PROF, N_LEVELS) float32 ...
    PSAL_CLMN            (N_PROF, N_LEVELS) float32 ...
    PSAL_CLSD            (N_PROF, N_LEVELS) float32 ...
    PSAL_ERME            (N_PROF, N_LEVELS) float32 ...
    PSAL_ERUR            (N_PROF, N_LEVELS) float32 ...
    PSAL_RESI            (N_PROF, N_LEVELS) float32 ...
Attributes:
    Conventions:       CF

In [5]:
print(type(int(a[1].values)))
print(str(int(a[150].values)))

<class 'int'>
1980


### TODO: It takes a lot of time to extract netcdf, find a way to save all TEMPERATURE LEVEL in DATAFRAME
see for example 

### Load all temperature netcdf files 

In [6]:
# Create function to preprocess the netdcf file in xarray
def cora_preproc_temp(ds):
    #SOURCE FILE
    SOURCE=np.empty(len(ds.N_PROF),dtype='S32')
    SOURCE[:]=ds.encoding['source'].split('/')[-1]
    ds['SOURCE']=xr.DataArray(SOURCE,dims='N_PROF')
    
    #TEMPERATURE LEVELS
    ds['TEMP0']=ds['TEMP'].isel(N_LEVELS=0) #TEMPERATURE SURFACE
    ds['TEMP1000']=ds['TEMP'].isel(N_LEVELS=101) #TEMPERATURE 1000m
    ds['TEMP2000']=ds['TEMP'].isel(N_LEVELS=151) #TEMPERATURE 2000m    
    
    #NO NEED VARIABLES
    ds=ds.drop(['REFERENCE_DATE_TIME','DATA_TYPE','DC_REFERENCE','DEPH','TEMP','TEMP_PROC','TEMP_QC','TEMP_CLMN','TEMP_CLSD','TEMP_ERME','TEMP_ERUR','TEMP_RESI'])
    
    # Add geographic selection:
    ds = ds.where((ds['LATITUDE'] > 20) & (ds['LATITUDE'] < 70) & (ds['LONGITUDE'] > -85) & (ds['LONGITUDE'] < 0) ,drop=True)
    
    #SQUEEZE
    ds=ds.squeeze()
    return ds

# XARRAY MFLOAD
CORA_temp=xr.open_mfdataset(root+'/'+yyyy+'/*TEMP.nc',decode_times=False,concat_dim='N_PROF',preprocess=cora_preproc_temp)
CORA_temp



/Users/locupe/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

/Users/locupe/anaconda2/envs/python3/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: Also `open_mfdataset` will no longer accept a `concat_dim` argument.
To get equivalent behaviour from now on please use the new
`combine_nested` function instead (or the `combine='nested'` option to
`open_mfdataset`).The datasets supplied do not have global dimension coordinates. In
future, to continue concatenating without supplying dimension
coordinates, please use the new `combine_nested` function (or the
`combine='nested'` option to open_mfdataset.
  from_openmfds=True,


<xarray.Dataset>
Dimensions:          (N_PROF: 71867)
Dimensions without coordinates: N_PROF
Data variables:
    PLATFORM_NUMBER  (N_PROF) object dask.array<chunksize=(5211,), meta=np.ndarray>
    WMO_INST_TYPE    (N_PROF) object dask.array<chunksize=(5211,), meta=np.ndarray>
    JULD             (N_PROF) float64 dask.array<chunksize=(5211,), meta=np.ndarray>
    LATITUDE         (N_PROF) float64 dask.array<chunksize=(5211,), meta=np.ndarray>
    LONGITUDE        (N_PROF) float64 dask.array<chunksize=(5211,), meta=np.ndarray>
    SOURCE           (N_PROF) object dask.array<chunksize=(5211,), meta=np.ndarray>
    TEMP0            (N_PROF) float32 dask.array<chunksize=(5211,), meta=np.ndarray>
    TEMP1000         (N_PROF) float32 dask.array<chunksize=(5211,), meta=np.ndarray>
    TEMP2000         (N_PROF) float32 dask.array<chunksize=(5211,), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.4
    title:             Monthly analysis
    institution:       OceanScope
    project_name:      CORA OA
    data_manager:      Tanguy Szekely
    software_version:  POSTOA_main - 7.0
    references:        Szekely et al. (2019)
    history:           20190227T090645L : Creation

### Convert array to Pandas DataFrame

In [7]:
#CONVERT ARRAY TO DATAFRAME
CORD_temp=CORA_temp.to_dataframe()

#JULD TO CALENDAR DATE
CORD_temp['DATE']=pd.to_datetime(CORD_temp['JULD'].values,unit='D',origin=pd.to_datetime('1950/1/1'))
CORD_temp=CORD_temp.drop(columns='JULD')

#BYTE STRING TO STRING
str_df = CORD_temp.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for col in str_df:
    CORD_temp[col] = str_df[col]
    
#HERE IS OUR PANDAS INDEX    
CORD_temp

,PLATFORM_NUMBER,WMO_INST_TYPE,LATITUDE,LONGITUDE,SOURCE,TEMP0,TEMP1000,TEMP2000,DATE
N_PROF,,,,,,,,,
0,IF000552,1023,46.702885,-2.290287,OA_CORA5.2_20180115_dat_TEMP.nc,11.107286,NaN,NaN,2017-12-16 00:42:11.250000000
1,IF000552,1023,46.702885,-2.290287,OA_CORA5.2_20180115_dat_TEMP.nc,10.780083,NaN,NaN,2017-12-23 21:42:11.250000000
2,IF000552,1023,46.702888,-2.290287,OA_CORA5.2_20180115_dat_TEMP.nc,10.302135,NaN,NaN,2017-12-29 13:49:41.250000000
3,FQBE,1023,48.616932,-4.697662,OA_CORA5.2_20180115_dat_TEMP.nc,14.076893,NaN,NaN,2017-12-06 14:20:37.500000000
4,FQBE,1023,48.789894,-4.316039,OA_CORA5.2_20180115_dat_TEMP.nc,11.668028,NaN,NaN,2017-12-18 05:26:15.000000000
...,...,...,...,...,...,...,...,...,...
71862,6901208,846,62.283100,-14.156700,OA_CORA5.2_20190115_dat_TEMP.nc,8.306000,5.364491,NaN,2019-02-23 11:52:25.999968000
71863,3901663,846,52.373001,-31.114000,OA_CORA5.2_20190115_dat_TEMP.nc,9.635000,3.806098,NaN,2019-01-22 20:09:48.000009600
71864,3901663,846,51.985001,-29.052999,OA_CORA5.2_20190115_dat_TEMP.nc,9.701000,3.787975,NaN,2019-02-01 17:02:24.000000000


### Load all salinity netcdf files and store them into dataframe

In [8]:
def cora_preproc_sal(ds):
    #SOURCE FILE
    SOURCE=np.empty(len(ds.N_PROF),dtype='S32')
    SOURCE[:]=ds.encoding['source'].split('/')[-1]
    ds['SOURCE']=xr.DataArray(SOURCE,dims='N_PROF')
    
    #PSALERATURE LEVELS
    ds['PSAL0']=ds['PSAL'].isel(N_LEVELS=0) #SALINITY SURFACE
    ds['PSAL1000']=ds['PSAL'].isel(N_LEVELS=101) #SALINITY 1000m
    ds['PSAL2000']=ds['PSAL'].isel(N_LEVELS=151) #SALINITY 2000m    
    
    #NO NEED VARIABLES
    ds=ds.drop(['REFERENCE_DATE_TIME','DATA_TYPE','DC_REFERENCE','DEPH','PSAL','PSAL_PROC','PSAL_QC','PSAL_CLMN','PSAL_CLSD','PSAL_ERME','PSAL_ERUR','PSAL_RESI'])
    
    # Add geographic selection:
    ds = ds.where((ds['LATITUDE'] > 20) & (ds['LATITUDE'] < 70) & (ds['LONGITUDE'] > -85) & (ds['LONGITUDE'] < 0) ,drop=True)
    
    #SQUEEZE
    ds=ds.squeeze()
    return ds

#XARRAY MFLOAD
CORA_psal=xr.open_mfdataset(root+'/'+yyyy+'/*PSAL.nc',decode_times=False,concat_dim='N_PROF',preprocess=cora_preproc_sal)
CORA_psal

#CONVERT ARRAY TO DATAFRAME
CORD_psal=CORA_psal.to_dataframe()

#JULD TO CALENDAR DATE
CORD_psal['DATE']=pd.to_datetime(CORD_psal['JULD'].values,unit='D',origin=pd.to_datetime('1950/1/1'))
CORD_psal=CORD_psal.drop(columns='JULD')

#BYTE STRING TO STRING
str_df = CORD_psal.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
for col in str_df:
    CORD_psal[col] = str_df[col]
    
#HERE IS OUR PANDAS INDEX    
CORD_psal

/Users/locupe/anaconda2/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

/Users/locupe/anaconda2/envs/python3/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: Also `open_mfdataset` will no longer accept a `concat_dim` argument.
To get equivalent behaviour from now on please use the new
`combine_nested` function instead (or the `combine='nested'` option to
`open_mfdataset`).The datasets supplied do not have global dimension coordinates. In
future, to continue concatenating without supplying dimension
coordinates, please use the new `combine_nested` function (or the
`combine='nested'` option to open_mfdataset.
  from_openmfds=True,


,PLATFORM_NUMBER,WMO_INST_TYPE,LATITUDE,LONGITUDE,SOURCE,PSAL0,PSAL1000,PSAL2000,DATE
N_PROF,,,,,,,,,
0,FQBE,1023,45.686054,-1.693818,OA_CORA5.2_20180115_dat_PSAL.nc,NaN,NaN,NaN,2017-12-06 00:42:11.250000000
1,FQBE,1023,46.264740,-1.693138,OA_CORA5.2_20180115_dat_PSAL.nc,NaN,NaN,NaN,2017-12-07 03:56:15.000000000
2,FQBE,1023,46.495636,-1.792024,OA_CORA5.2_20180115_dat_PSAL.nc,NaN,NaN,NaN,2017-12-18 05:26:15.000000000
3,FMLW,1023,49.045464,-3.997547,OA_CORA5.2_20180115_dat_PSAL.nc,NaN,NaN,NaN,2017-12-18 23:20:37.500000000
4,XJBI,1023,49.138432,-63.489079,OA_CORA5.2_20180115_dat_PSAL.nc,NaN,NaN,NaN,2017-12-05 02:37:30.000000000
...,...,...,...,...,...,...,...,...,...
79594,6901208,846,61.941799,-17.236000,OA_CORA5.2_20181215_dat_PSAL.nc,35.117001,34.948582,NaN,2018-12-16 18:56:34.999958400
79595,6901208,846,62.106300,-16.271299,OA_CORA5.2_20181215_dat_PSAL.nc,35.105999,34.960060,NaN,2018-12-26 14:55:58.999987200
79596,6901208,846,62.532501,-17.319500,OA_CORA5.2_20181215_dat_PSAL.nc,35.090000,34.945034,NaN,2019-01-05 10:42:37.999987200


In [9]:
# MERGE temp and psal dataframes?

### Export dataframe in csv files and save them on laptop and external hard-drive

In [13]:
pathtodatafile1 = root+'/'+'index/'
pathtodatafile2 = '~/Dropbox/Work/Python/Repos_perso/explore_cora/data/index_CORA_TS_OA_REP/'

CORD_temp.to_csv(pathtodatafile1+'CORA_TS_OA_REP_NATL_temp_'+yyyy+'.csv')
CORD_psal.to_csv(pathtodatafile1+'CORA_TS_OA_REP_NATL_psal_'+yyyy+'.csv')

CORD_temp.to_csv(pathtodatafile2+'CORA_TS_OA_REP_NATL_temp_'+yyyy+'.csv')
CORD_psal.to_csv(pathtodatafile2+'CORA_TS_OA_REP_NATL_psal_'+yyyy+'.csv')

### Delete directory of extracted files to free some space
Delete the directory created at the beginning of the notebook to free some space (the decompressed version of the original CORA files takes up to ~100 times more space that the compressed version!)

In [14]:
# NOT NEEDED AS THE ARCHIVE IS NOT DELETED AFTER EXTRACTION
# # Compression of all directory into original archive files (CORA-5.2-data-YYYY.tgz)
# dirpath = root+'/'

# tgzfile = 'CORA-5.2-data-'+yyyy+'.tgz'

# tar = tarfile.open(dirpath+tgzfile, "w:gz")
# tar.add(dirpath+yyyy)
# tar.close()

# Delete directory of extracted files to free some space
import shutil

mydir = root+'/'+yyyy
try:
    shutil.rmtree(mydir)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))




### THINGS TO DO:
 -> Adapt this for the INSITU_GLO_TS_REP_OBSERVATIONS_013_001_b database (https://www.seanoe.org/data/00351/46219/)